In [1]:
import json
import os.path as ops
import numpy as np
import torch
import cv2
import time
import os
import matplotlib.pylab as plt
import sys
from tqdm import tqdm
sys.path.append('..')
from dataset.dataset_utils import TUSIMPLE
from Lanenet.model2 import Lanenet
from utils.evaluation import gray_to_rgb_emb, process_instance_embedding, video_to_clips

# Load the Model

In [2]:
model_path = './TUSIMPLE/Lanenet_output/lanenet_epoch_22_batch_8.model'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
LaneNet_model = Lanenet(2, 4)
LaneNet_model.load_state_dict(torch.load(model_path))
LaneNet_model.to(device)
print('success')

success


# Load the Test Dataset

Test use TUSIMPLE test dataset ``clips`` and ``test_label.json`` and write the predit result in ``test_tasks_0627.json`` use the evaluation from TUSIMPLE dataset in ``utils/lane.py``

write lanes and run_time to ``pred_result.json``

In [5]:
pred_json_path = './TUSIMPLE/train_set/labels/train_labels0.json'
json_pred = [json.loads(line) for line in open(pred_json_path).readlines()]

In [6]:
all_time_forward = []
all_time_clustering = []
for i, sample in enumerate(tqdm(json_pred)):
    h_samples = sample['h_samples']
    lanes = sample['lanes']
    run_time = 1000
    raw_file = sample['raw_file']
    img_path = ops.join('./TUSIMPLE/train_set', raw_file)
    # read the image
    gt_img_org = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    org_shape = gt_img_org.shape
    gt_image = cv2.resize(gt_img_org, dsize=(512, 256), interpolation=cv2.INTER_LINEAR)
    gt_image = gt_image / 127.5 - 1.0
    gt_image = torch.tensor(gt_image, dtype=torch.float)
    gt_image = np.transpose(gt_image, (2, 0, 1))
    # Go through the network
    time_start=time.time()
    binary_final_logits, instance_embedding = LaneNet_model(gt_image.unsqueeze(0).cuda())
    # binary_final_logits = binary_final_logits.cpu()
    # instance_embedding = instance_embedding.cpu()
    time_end=time.time()
    # Get the final embedding image
    binary_img = torch.argmax(binary_final_logits, dim=1).squeeze().cpu().numpy()
    binary_img[0:50,:] = 0
    clu_start = time.time()
    rbg_emb, cluster_result = process_instance_embedding(instance_embedding.cpu(), binary_img,
                                                             distance=1.5, lane_num=4)
    clu_end = time.time()
    cluster_result = cv2.resize(cluster_result, dsize=(org_shape[1], org_shape[0]), 
                                interpolation=cv2.INTER_NEAREST)
    elements = np.unique(cluster_result)
    for line_idx in elements:
        if line_idx == 0:
            continue
        else:
            mask = (cluster_result == line_idx)
            select_mask = mask[h_samples]
            row_result = []
            for row in range(len(h_samples)):
                col_indexes = np.nonzero(select_mask[row])[0]
                if len(col_indexes) == 0:
                    row_result.append(-2)
                else:
                    row_result.append(int(col_indexes.min() + (col_indexes.max()-col_indexes.min())/2))
            json_pred[i]['lanes'].append(row_result)
            json_pred[i]['run_time'] = time_end-time_start
            all_time_forward.append(time_end-time_start)
            all_time_clustering.append(clu_end-clu_start)

  0%|          | 0/50 [00:00<?, ?it/s]


KeyError: 'run_time'

In [7]:
forward_avg = np.sum(all_time_forward[500:2000])/1500
cluster_avg = np.sum(all_time_clustering[500:2000])/1500

print('The Forward pass time for one image is: {}ms'.format(forward_avg*1000))
print('The Clustering time for one image is: {}ms'.format(cluster_avg*1000))
print('The total time for one image is: {}ms'.format((cluster_avg+forward_avg)*1000))

print('The speed for foreard pass is: {}fps'.format(1/forward_avg))
print('The speed for foreard pass is: {}fps'.format(1/cluster_avg))

The Forward pass time for one image is: 0.0ms
The Clustering time for one image is: 0.0ms
The total time for one image is: 0.0ms
The speed for foreard pass is: inffps
The speed for foreard pass is: inffps


/home/sutchu/PycharmProjects/lanenet_pytorch/venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  
/home/sutchu/PycharmProjects/lanenet_pytorch/venv/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':


In [8]:
with open('../TUSIMPLE/pred.json', 'w') as f:
    for res in json_pred:
        json.dump(res, f)
        f.write('\n')

# Evaluation using TUSIMPLE script

In [59]:
from utils.lane import LaneEval

In [30]:
result = LaneEval.bench_one_submit('../TUSIMPLE/pred.json',
                         '../TUSIMPLE/test_set/test_label.json')

In [31]:
print(result)

[{"name":"Accuracy","value":0.9427976190476187,"order":"desc"},{"name":"FP","value":0.12833333333333333,"order":"asc"},{"name":"FN","value":0.08833333333333333,"order":"asc"}]


# Evaluation for aug result

In [44]:
model_path = '../TUSIMPLE/Lanenet_output/lanenet_epoch_39_batch_8_AUG.model'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
LaneNet_model = Lanenet(2, 4)
LaneNet_model.load_state_dict(torch.load(model_path, map_location=torch.device('cuda')))
LaneNet_model.to(torch.device('cpu'))
print('success')

success


In [45]:
pred_json_path = '../TUSIMPLE/test_set/test_tasks_0627.json'
json_pred = [json.loads(line) for line in open(pred_json_path).readlines()]

In [46]:
all_time_forward = []
all_time_clustering = []
for i, sample in enumerate(tqdm(json_pred)):
    h_samples = sample['h_samples']
    lanes = sample['lanes']
    run_time = sample['run_time']
    raw_file = sample['raw_file']
    img_path = ops.join('../TUSIMPLE/test_set', raw_file)
    # read the image
    gt_img_org = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    org_shape = gt_img_org.shape
    gt_image = cv2.resize(gt_img_org, dsize=(512, 256), interpolation=cv2.INTER_LINEAR)
    gt_image = gt_image / 127.5 - 1.0
    gt_image = torch.tensor(gt_image, dtype=torch.float)
    gt_image = np.transpose(gt_image, (2, 0, 1))
    # Go through the network
    time_start=time.time()
    binary_final_logits, instance_embedding = LaneNet_model(gt_image.unsqueeze(0))
    # binary_final_logits = binary_final_logits.cpu()
    # instance_embedding = instance_embedding.cpu()
    time_end=time.time()
    # Get the final embedding image
    binary_img = torch.argmax(binary_final_logits, dim=1).squeeze().cpu().numpy()
    binary_img[0:50,:] = 0
    clu_start = time.time()
    rbg_emb, cluster_result = process_instance_embedding(instance_embedding.cpu(), binary_img,
                                                             distance=1.5, lane_num=4)
    clu_end = time.time()
    cluster_result = cv2.resize(cluster_result, dsize=(org_shape[1], org_shape[0]), 
                                interpolation=cv2.INTER_NEAREST)
    elements = np.unique(cluster_result)
    for line_idx in elements:
        if line_idx == 0:
            continue
        else:
            mask = (cluster_result == line_idx)
            select_mask = mask[h_samples]
            row_result = []
            for row in range(len(h_samples)):
                col_indexes = np.nonzero(select_mask[row])[0]
                if len(col_indexes) == 0:
                    row_result.append(-2)
                else:
                    row_result.append(int(col_indexes.min() + (col_indexes.max()-col_indexes.min())/2))
            json_pred[i]['lanes'].append(row_result)
            json_pred[i]['run_time'] = time_end-time_start
            all_time_forward.append(time_end-time_start)
            all_time_clustering.append(clu_end-clu_start)

100%|██████████| 50/50 [00:53<00:00,  1.08s/it]


In [47]:
with open('../TUSIMPLE/pred_aug.json', 'w') as f:
    for res in json_pred:
        json.dump(res, f)
        f.write('\n')

In [48]:
from utils.lane import LaneEval

In [49]:
result = LaneEval.bench_one_submit('../TUSIMPLE/pred_aug.json',
                         '../TUSIMPLE/test_set/test_label.json')

In [50]:
print(result)

[{"name":"Accuracy","value":0.9372916666666664,"order":"desc"},{"name":"FP","value":0.14166666666666666,"order":"asc"},{"name":"FN","value":0.10499999999999998,"order":"asc"}]
